<a href="https://colab.research.google.com/github/JainamS1996/Firefly-Algorithm/blob/master/Firefly_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import operator


from numpy.random import random_sample

__all__ = ['Ackley', 'Michalewicz']


class BaseFunc:
    def __init__(self, dim):
        self.dim = dim
        self.min_bound = np.zeros(self.dim)
        self.max_bound = np.zeros(self.dim)
        self.solution = np.zeros(self.dim)
        self.global_optima = 0
        self.plot_place = 0.25
        self.m = 10
        self.title = ''

    def get_global_optima(self):
        return self.global_optima

    def get_solution(self):
        return self.solution

    def get_search_bounds(self):
        return [self.min_bound, self.max_bound]

    def get_y(self, x):
        return -1

    def plot(self):
        x = np.arange(self.min_bound[0], self.max_bound[0], self.plot_place, dtype=np.float32)
        y = np.arange(self.min_bound[1], self.max_bound[1], self.plot_place, dtype=np.float32)
        X, Y = np.meshgrid(x, y)
        Z = []
        for coord in zip(X, Y):
            z = []
            for input in zip(coord[0], coord[1]):
                tmp = list(input)
                tmp.extend(list(self.solution[0:self.dim - 2]))
                z.append(self.get_y(np.array(tmp)))
            Z.append(z)
        Z = np.array(Z)
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.plot_wireframe(X, Y, Z)
        plt.show()


class Ackley(BaseFunc):

    def __init__(self, dim):
        super().__init__(dim)
        self.max_bound = np.array([32.768] * self.dim)
        self.min_bound = np.array([-32.768] * self.dim)
        self.solution = np.ones(self.dim)
        self.global_optima = 0
        self.title = 'Ackley'

    def get_y(self, x):
        return 20. - 20. * np.exp(-0.2 * np.sqrt(1. / self.dim * np.sum(np.square(x)))) + np.e - np.exp(
            1. / self.dim * np.sum(np.cos(x * 2. * np.pi)))

    def get_y_2d(self, x, y):
        return 20. - 20. * np.exp(-0.2 * np.sqrt(1. / self.dim * (x ** 2 + y ** 2))) + np.e - np.exp(
            1. / self.dim * (np.cos(x * 2. * np.pi) + np.cos(y * 2. * np.pi)))


class Michalewicz(BaseFunc):

    def __init__(self, dim):
        super().__init__(dim)
        self.max_bound = np.array([np.pi] * self.dim)
        self.min_bound = np.zeros(self.dim)
        self.solution = np.zeros(self.dim)
        self.global_optima = self.get_y(self.solution)
        self.title = 'Michalewicz'
        self.m = 10

    def get_y(self, x):
        y = 0
        for i in range(self.dim):
            y += np.sin(x[i]) * np.power(np.sin((i + 1) * np.power(x[i], 2) / np.pi), 2 * self.m)
        return -y

    def get_y_2d(self, x, y):
        yy = 0
        yy += np.sin(x) * np.power(np.sin((0 + 1) * np.power(x, 2) / np.pi), 2 * self.m)
        yy += np.sin(y) * np.power(np.sin((1 + 1) * np.power(y, 2) / np.pi), 2 * self.m)
        return -yy

    
def generate_population(population_size, problem_dim, min_bound, max_bound):
    predictedY = ga_model.predict(trainx)
    error = np.mean((trainy_binary - predictedY)**2)
    data = (max_bound + error - min_bound) * random_sample((population_size, problem_dim)) + min_bound
    data[data > max_bound] = max_bound
    return data
    
# TODO: refactor function firefly_optimisation function call
class Firefly:

    def __init__(self, problem_dim, min_bound, max_bound):
        self.func = Michalewicz(problem_dim)
        self.position = generate_population(1, problem_dim, min_bound, max_bound)[0]
        self.brightness = None
        self.update_brightness()

    # the best fit is 0
    def update_brightness(self):
        self.brightness = -self.func.get_y(self.position)


class FireflyOptimizer:

    def __init__(self, **kwargs):
        self.population_size = int(kwargs.get('population_size', 10))
        self.problem_dim = kwargs.get('problem_dim', 2)
        self.min_bound = kwargs.get('min_bound', -5)
        self.max_bound = kwargs.get('max_bound', 5)
        self.generations = kwargs.get('generations', 10)
        self.population = self._population(self.population_size, self.problem_dim, self.min_bound, self.max_bound)
        self.gamma = kwargs.get('gamma', 0.97)  # absorption coefficient
        self.alpha = kwargs.get('alpha', 0.25)  # randomness [0,1]
        self.beta_init = kwargs.get('beta_init', 1)
        self.beta_min = kwargs.get('beta_min', 0.2)
        self.optimization_benchmark = kwargs.get('optimization_benchmark', 'Ackley')
    
    
    @staticmethod
    
    def _population(population_size, problem_dim, min_bound, max_bound):
        population = []
        for i in range(population_size):
            population.append(Firefly(problem_dim, min_bound, max_bound))
        return population

    def step(self):
        self.population.sort(key=operator.attrgetter('brightness'), reverse=True)
        self._modify_alpha()
        tmp_population = self.population
        for i in range(self.population_size):
            for j in range(self.population_size):
                if self.population[i].brightness > tmp_population[j].brightness:
                    r = math.sqrt(np.sum((self.population[i].position - tmp_population[j].position) ** 2))
                    beta = (self.beta_init - self.beta_min) * math.exp(-self.gamma * r ** 2) + self.beta_min
                    tmp = self.alpha * (np.random.random_sample((1, self.problem_dim))[0] - 0.5) * (
                            self.max_bound - self.min_bound)
                    self.population[j].position = self.check_position(
                        self.population[i].position * (1 - beta) + tmp_population[
                            j].position * beta + tmp)
                    self.population[j].update_brightness()
        self.population[0].position = generate_population(1, self.problem_dim, self.min_bound, self.max_bound)[0]
        self.population[0].update_brightness()

    def run_firefly(self):
        for t in range(self.generations):
            print('Generation %s, best fitness %s' % (t, self.population[0].brightness))
            self.step()
        self.population.sort(key=operator.attrgetter('brightness'), reverse=True)
        a[]=self.population[0].brightness
        return self.population[0].brightness, self.population[0].position
    def check_position(self, position):
        position[position > self.max_bound] = self.max_bound
        position[position < self.min_bound] = self.min_bound
        return position

    def _modify_alpha(self):
        delta = 1 - (10 ** (-4) / 0.9) ** (1 / self.generations)
        self.alpha = (1 - delta) * self.alpha

In [0]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split

from keras.layers import LSTM, Input, Dense
from keras.models import Model
from keras.models import Sequential

from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Activation
from keras import backend
from keras import backend as K
from keras.optimizers import SGD
from keras.utils import to_categorical




In [0]:
df = pd.read_csv('clean_bank.csv',sep=',',header=0)
df.shape
df = df.iloc[4000:]
df.shape

(521, 14)

In [0]:
x=df.iloc[:,1:13]
y=df.iloc[:,13]
trainx,testx,trainy,testy=split(x,y,test_size=0.33)
print(trainx.shape,testx.shape)

(349, 12) (172, 12)


In [0]:
trainx=trainx.values
trainy=trainy.values

In [0]:
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=12,name='layer_1'))
model.add(Dense(64, activation='relu',name='layer_2'))
model.add(Dense(2, activation='softmax',name='layer_3'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(trainx, trainy_binary,
          epochs=100,
          batch_size=128)
score = model.evaluate(testx, testy_binary, batch_size=128)
model.summary()

Epoch 1/100
349/349 [==============================] - 0s 1ms/step - loss: 1.8916 - acc: 0.8682
Epoch 2/100
349/349 [==============================] - 0s 30us/step - loss: 1.7088 - acc: 0.8940
Epoch 3/100
349/349 [==============================] - 0s 45us/step - loss: 1.7088 - acc: 0.8940
Epoch 4/100
349/349 [==============================] - 0s 39us/step - loss: 1.7088 - acc: 0.8940
Epoch 5/100
349/349 [==============================] - 0s 49us/step - loss: 1.7088 - acc: 0.8940
Epoch 6/100
349/349 [==============================] - 0s 41us/step - loss: 1.7088 - acc: 0.8940
Epoch 7/100
349/349 [==============================] - 0s 49us/step - loss: 1.7088 - acc: 0.8940
Epoch 8/100
349/349 [==============================] - 0s 51us/step - loss: 1.7088 - acc: 0.8940
Epoch 9/100
349/349 [==============================] - 0s 39us/step - loss: 1.7088 - acc: 0.8940
Epoch 10/100
349/349 [==============================] - 0s 36us/step - loss: 1.7088 - acc: 0.8940
Epoch 11/100
349/349 [========

In [0]:
trainy_binary = to_categorical(trainy)
testy_binary = to_categorical(testy)
print(trainy_binary)

In [0]:
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

def r_square_loss(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - ( 1 - SS_res/(SS_tot + K.epsilon()))

In [0]:
print(score)

[2.4364563531653824, 0.848837207916171]


In [0]:
weights_layer1 = model.layers[0].get_weights()[0]
biases_layer1 = model.layers[0].get_weights()[1]

weights_layer2 = model.layers[1].get_weights()[0]
biases_layer2 = model.layers[1].get_weights()[1]

weights_layer3 = model.layers[2].get_weights()[0]
biases_layer3 = model.layers[2].get_weights()[1]

print(np.shape(weights_layer1))
print(np.shape(biases_layer1))
print(np.shape(weights_layer2))
print(np.shape(biases_layer2))
print(np.shape(weights_layer3))
print(np.shape(biases_layer3))

(12, 64)
(64,)
(64, 64)
(64,)
(64, 2)
(2,)


In [0]:
ga_model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
ga_model.add(Dense(64, activation='relu', input_dim=12,name='layer_1'))
ga_model.add(Dense(64, activation='relu',name='layer_2'))
ga_model.add(Dense(2, activation='softmax',name='layer_3'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
ga_model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
first_layer = ga_model.get_layer('layer_1').get_weights()
second_layer = ga_model.get_layer('layer_2').get_weights()
third_layer=ga_model.get_layer('layer_3').get_weights()
dimensions = (first_layer[0].shape[0] * first_layer[0].shape[1]) + (first_layer[1].shape[0]) + (second_layer[0].shape[0] * second_layer[0].shape[1]) +(second_layer[1].shape[0])+(third_layer[0].shape[0] * third_layer[0].shape[1]) +(third_layer[1].shape[0])
print("Number of params in Neural Network: %d" % dimensions)

optimizer = FireflyOptimizer(population_size=25, problem_dim=dimensions, generations=200, beta_min=0.65, alpha=0.05)
optimizer.run_firefly()


Number of params in Neural Network: 5122
Generation 0, best fitness -85.16611768182382
Generation 1, best fitness -60.9299954418877
Generation 2, best fitness -53.943469943582144
Generation 3, best fitness -58.069283445749065
Generation 4, best fitness -88.36110396182877
Generation 5, best fitness -78.13870230602026
Generation 6, best fitness -102.35557685825097
Generation 7, best fitness -69.25458886169973
Generation 8, best fitness -53.222446460770826
Generation 9, best fitness -63.54054883588625
Generation 10, best fitness -75.46930634574544
Generation 11, best fitness -49.771603513182875
Generation 12, best fitness -66.48247069978305
Generation 13, best fitness -91.01380954661145
Generation 14, best fitness -52.99090983232552
Generation 15, best fitness -73.98690333687217
Generation 16, best fitness -88.34384950092813
Generation 17, best fitness -70.69725608713978
Generation 18, best fitness -100.92782871497074
Generation 19, best fitness -48.69892350902817
Generation 20, best fitn

(266.78009811265144,
 array([ 0.32725945,  0.28874117, -0.36118012, ...,  0.41926962,
        -0.35005179,  0.54773039]))

In [0]:
''''import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


__all__ = ['Ackley', 'Michalewicz']


class BaseFunc:
    def __init__(self, dim):
        self.dim = dim
        self.min_bound = np.zeros(self.dim)
        self.max_bound = np.zeros(self.dim)
        self.solution = np.zeros(self.dim)
        self.global_optima = 0
        self.plot_place = 0.25
        self.m = 10
        self.title = ''

    def get_global_optima(self):
        return self.global_optima

    def get_solution(self):
        return self.solution

    def get_search_bounds(self):
        return [self.min_bound, self.max_bound]

    def get_y(self, x):
        return -1

    def plot(self):
        x = np.arange(self.min_bound[0], self.max_bound[0], self.plot_place, dtype=np.float32)
        y = np.arange(self.min_bound[1], self.max_bound[1], self.plot_place, dtype=np.float32)
        X, Y = np.meshgrid(x, y)
        Z = []
        for coord in zip(X, Y):
            z = []
            for input in zip(coord[0], coord[1]):
                tmp = list(input)
                tmp.extend(list(self.solution[0:self.dim - 2]))
                z.append(self.get_y(np.array(tmp)))
            Z.append(z)
        Z = np.array(Z)
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.plot_wireframe(X, Y, Z)
        plt.show()


class Ackley(BaseFunc):

    def __init__(self, dim):
        super().__init__(dim)
        self.max_bound = np.array([32.768] * self.dim)
        self.min_bound = np.array([-32.768] * self.dim)
        self.solution = np.ones(self.dim)
        self.global_optima = 0
        self.title = 'Ackley'

    def get_y(self, x):
        return 20. - 20. * np.exp(-0.2 * np.sqrt(1. / self.dim * np.sum(np.square(x)))) + np.e - np.exp(
            1. / self.dim * np.sum(np.cos(x * 2. * np.pi)))

    def get_y_2d(self, x, y):
        return 20. - 20. * np.exp(-0.2 * np.sqrt(1. / self.dim * (x ** 2 + y ** 2))) + np.e - np.exp(
            1. / self.dim * (np.cos(x * 2. * np.pi) + np.cos(y * 2. * np.pi)))


class Michalewicz(BaseFunc):

    def __init__(self, dim):
        super().__init__(dim)
        self.max_bound = np.array([np.pi] * self.dim)
        self.min_bound = np.zeros(self.dim)
        self.solution = np.zeros(self.dim)
        self.global_optima = self.get_y(self.solution)
        self.title = 'Michalewicz'
        self.m = 10

    def get_y(self, x):
        y = 0
        for i in range(self.dim):
            y += np.sin(x[i]) * np.power(np.sin((i + 1) * np.power(x[i], 2) / np.pi), 2 * self.m)
        return -y

    def get_y_2d(self, x, y):
        yy = 0
        yy += np.sin(x) * np.power(np.sin((0 + 1) * np.power(x, 2) / np.pi), 2 * self.m)
        yy += np.sin(y) * np.power(np.sin((1 + 1) * np.power(y, 2) / np.pi), 2 * self.m)
        return -yy